In [3]:
import numpy as np
import sklearn.linear_model as lm
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns # a visualization lib based on matplotlib, more convenient and beautiful

In [97]:
df = pd.read_csv('dataport-export_gas_oct2015-mar2016.csv')
print(df.head())
print(f'The indices are: {df.columns}')

localminute  dataid  meter_value
0  2015-10-01 00:00:10-05     739        88858
1  2015-10-01 00:00:13-05    8890       197164
2  2015-10-01 00:00:20-05    6910       179118
3  2015-10-01 00:00:22-05    3635       151318
4  2015-10-01 00:00:22-05    1507       390354
The indices are: Index(['localminute', 'dataid', 'meter_value'], dtype='object')


1.1 How many houses are included in the measurement study? Are there any malfunctioning meters? If so, identify them and the time periods where they were malfunctioning. The information below regarding data collection may be useful.

157 houses

Many types of malfunctioning meters: 

1. Since the homes in this neighborhood are primarily newly constructed (including single-family homes, apartments, and town homes), the meters first occur after 2015-10-01 with large meter values may malfunction or just the case that the are newly move in this house.

2. the meter reading decreases since time goes by.

3. the meter reading increase a lot in a short time period.

4. the meter doesn't report to the meter data management center (MDMC).




1.1.1 the first case

get the first record of each home and a list of dataid for further analysis.

last 37 meters in df_start may be malfunctional.



In [98]:
df_start = df.drop_duplicates(subset='dataid',keep='first',inplace=False)
df_end = df.drop_duplicates(subset='dataid',keep='last',inplace=False)
# get a list of meterID
homeid = list(df_start['dataid'])
print("Total number of meters is :", len(df_start))
if not df.isnull().values.any():
    print("There is no data missing or 'NaN'")
print("Check the report time for the last 40 meters:" )
print(df_start.tail(40))

Total number of meters is : 157
There is no data missing or 'NaN'
Check the report time for the last 40 meters:
                           localminute  dataid  meter_value
4478            2015-10-01 12:47:44-05    2378       159376
4931            2015-10-01 14:10:29-05    6830       150386
6966            2015-10-01 20:04:43-05    9766       158512
11436           2015-10-02 08:54:27-05    4193       289668
18211           2015-10-03 03:48:54-05    8703       210416
18597           2015-10-03 04:51:53-05    6578       157806
35710           2015-10-05 02:44:01-05    4228        85162
47472           2015-10-06 11:52:20-05    2645        59318
63854           2015-10-08 10:01:28-05    6505       132928
63993           2015-10-08 10:22:34-05    5395       136658
66974           2015-10-08 18:13:29-05    8059       101626
82535           2015-10-10 11:54:39-05    7965       174608
101655          2015-10-12 18:33:44-05      44       165674
109298          2015-10-13 14:50:40-05    4671  

In [91]:

dec_change = pd.DataFrame(columns=['localminute','dataid','meter_value'])
inc_huge = pd.DataFrame(columns=['localminute','dataid','meter_value'])
stop_report = pd.DataFrame(columns=['localminute','dataid','meter_value'])
for i in homeid:
    # print(i)
    df_home = df[df['dataid']==i]
    readings = list(df_home['meter_value'])
    time = list(df['localminute'])
    
    for j in range(len(df_home)-1):
        if (readings[j+1] - readings[j]) < 0:
            dec_change = dec_change.append(df_home.iloc[j:j+2])
        if (readings[j+1] - readings[j]) >= 200:
            inc_huge = inc_huge.append(df_home.iloc[j:j+2])
            # print(df_home.iloc[j:j+2])
        hour_1 = int(time[j].split(':')[0][-2:])
        hour_2 = int(time[j+1].split(':')[0][-2:])
        day_1 = int(time[j].split('-')[0][:2])
        day_2 = int(time[j+1].split('-')[0][:2])
        month_1 = int(time[j].split('-')[1])
        month_2 = int(time[j+1].split('-')[1])
        if month_2-month_1>1 or day_2-day_1>1 or hour_2-hour_1>1:
            stop_report = stop_report.append(df_home.iloc[j:j+2])



1.1.2. the meter reading decreases since time goes by.

check each meter and return the meter_id and time stamp

In [56]:
dec_start = dec_change.drop_duplicates(subset='dataid',keep='first',inplace=False)
# dec_id = list(dec_start['dataid'])
print(f"\nIn total, {len(dec_start)} meters have decreasing values.")
print("\nHere are the meters that occurs decreasing value and the corresponding times.")
dec_times = dec_change.pivot_table(index=['dataid'],aggfunc='size')
print('\n',(dec_times/2).astype('int'))


In total, 43 meters have decreasing values.

Here are the meters that occurs decreasing value and the corresponding times.

 dataid
35        1
77        1
94        6
483       1
484       9
1042      1
1086      1
1185    135
1507      2
1556     12
1718      4
1790      1
1801      1
2129      3
2335      5
2449     93
3134     18
3527      1
3544     18
3893      2
4031     16
4193      1
4514    141
4998      1
5129     76
5131      1
5193      4
5403    156
5810     10
5814      1
5892      1
6836     51
7017      1
7030     90
7117    123
7739      1
7794      1
7989      2
8156    151
8890     44
9134    115
9639      2
9982      2
dtype: int64


Consider more than 20 times is malfunction. Take a look at 10 samples of "1185"

In [45]:
dec_exa = dec_change[dec_change['dataid']==1185]
dec_exa.head(20)

,localminute,dataid,meter_value
143717,2015-10-17 10:50:54-05,1185,139798
143720,2015-10-17 10:51:09-05,1185,139796
143782,2015-10-17 10:59:49-05,1185,139798
143836,2015-10-17 11:08:06-05,1185,139796
144068,2015-10-17 11:43:41-05,1185,139800
144087,2015-10-17 11:47:03-05,1185,139798
144386,2015-10-17 12:31:41-05,1185,139806
144399,2015-10-17 12:33:57-05,1185,139802
144586,2015-10-17 13:03:51-05,1185,139812
144745,2015-10-17 13:27:01-05,1185,139806


1.1.3. the meter reading increase a lot in a short time period.

Take larger that 200 cubics increase between two adjacent records as an outlier.


In [46]:
inc_start = inc_huge.drop_duplicates(subset='dataid',keep='first',inplace=False)
print(f"\n There are {len(inc_start)} meters has outlier.")
print("\n Take a brief look at 10 outliers of some meters.")
inc_huge.head(20)


 There are 93 meters has outlier.

 Take a brief look at 10 outliers of one meter.}


,localminute,dataid,meter_value
609165,2015-12-07 23:41:42-06,3635,155548
618896,2015-12-09 00:26:36-06,3635,155788
732234,2015-12-21 00:29:14.700727-06,4352,223624
1132977,2016-02-04 13:18:31.172764-06,4352,233896
1154982,2016-02-06 22:15:47.473284-06,4352,234414
1159974,2016-02-07 11:23:17.30182-06,4352,234672
1422869,2016-03-08 18:09:51.535581-06,4352,236952
1464967,2016-03-14 18:47:32.658793-05,4352,237172
1491179,2016-03-18 11:27:23.980261-05,4352,237230
1515940,2016-03-21 21:41:15.66264-05,4352,237674


Apparently, there are two kinds of outliers.

One is report period is much longer, but the value make sense.

The other one may have two different values (don't know how to describe this), such as "8156". Half of the data like this will be dropped for further analysis.


In [47]:
outliers = pd.DataFrame(columns=['localminute','dataid','meter_value'])
for i in homeid:
    # print(i)
    df_home = df[df['dataid']==i]
    readings = list(df_home['meter_value'])

    for j in range(len(df_home)-1):
        if (readings[j+1] - readings[j]) >= 2000:
            outliers = outliers.append(df_home.iloc[j:j+2])

In [68]:
out_start = outliers.drop_duplicates(subset='dataid',keep='first',inplace=False)
out_times = outliers.pivot_table(index=['dataid'],aggfunc='size')
print('\n Here are the meters that occurs outliers and the corresponding times.')
print('\n', (out_times/2).astype(int))



 Here are the meters that occurs outliers and the corresponding times.

 dataid
222       5
1185    125
1556     10
2233      2
2335      4
2449     91
2638      1
2645      1
3039      1
3134     17
3544     18
4352      1
4421      1
4447      1
4514    126
4671      1
4874      1
5129     73
5403    136
5545      2
6505      1
6685      3
6836     50
6863      2
7030     79
7117    111
7460      1
7919      1
7965      1
8156    139
8467      1
8703      2
9134    111
9474      1
9620      4
9639      2
9956      1
9982      2
dtype: int64


After a brief look, half of the outliers should be dropped for meter ID:

1185, 1556, 2449, 3134, 3544, 4514, 5129, 5403, 6836, 7030, 7117, 8156, 9134

1.1.4 the meter doesn't report to the meter data management center (MDMC).

In [130]:
if stop_report.any() is True:
    print(len(stop_report))
else:
    print("There isn't any meter reporting nothing in a time period longer than one hour.")

There isn't any meter reporting nothing in a time period longer than one hour.
